<a href="https://colab.research.google.com/github/swathysv/TensorFlow/blob/master/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import datetime
import os
import numpy as np
import imutils

In [ ]:
face_haar_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#Place the xml file in the same path as this notebook or pass the absolute path of the xml file

In [ ]:
def faceDetection(img):
    gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_haar_cascade.detectMultiScale(gray_img,scaleFactor=1.32,minNeighbors=5)
    #faces are the coordinates of the roi
    return faces,gray_img

In [ ]:
capture=cv2.VideoCapture(0) #0- for camera. To use any video as input, pass the video file as the parameter
print(capture.isOpened)
while(capture.isOpened):
    captured,frame=capture.read()
    
    faces_detected,gray_img=faceDetection(frame)

    for (x,y,w,h) in faces_detected:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),thickness=3)
    
    cv2.imshow("Web Cam",frame)    
    if cv2.waitKey(1)&0xff==ord('q'): #Press 'q' to close the window
        break
capture.release()
cv2.destroyAllWindows()

<built-in method isOpened of cv2.VideoCapture object at 0x00000124E0B279D0>


In [ ]:
#give labels as numbers to the training data. Folder name is the label itself
#function returns region of interest(roi) and the label of images 
def lable_training_data(directory):
    faces=[]
    labels=[]
    for path , sub_dir, filenames in os.walk(directory):
        for filename in filenames:
            if filename.startswith("."):
                print("skipping system file"+filename)
                continue
            #gives folder name in which file is present
            label=os.path.basename(path)
            img_path=os.path.join(path,filename)
            print("img_path-",img_path)
            print("label-",label)
            test_img=cv2.imread(img_path)
            if test_img is None:
                print("couldnt load image properly")
                continue
            faces_rect,gray_img=faceDetection(test_img)
            if len(faces_rect)!=1:
                #skipping images with multiple images
                print("Skipping image"+img_path)
                continue
            (x,y,w,h)=faces_rect[0]
            roi_gray=gray_img[y:y+w,x:x+h]
            faces.append(roi_gray)
            labels.append(int(label))
    return faces,labels
            

In [ ]:
faces,labels=lable_training_data("C:\\Users\\FaceRecognition\\Data")
#Pass the path of the folder where all your labelled folders are present
#In this case, "Data" Folder should consist of folders "1","2","3",.. each containing your training Data.
#You can use publicly available datasets for training

In [ ]:
def train_classifier(faces,labels):
#Local Binary Pattern Histogram
    face_recognizer=cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(faces,np.array(labels))
    return face_recognizer
    

In [ ]:
face_recognizer=train_classifier(faces,labels)

In [ ]:
def draw_rect(test_img,face):
    (x,y,w,h)=face
    cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=4)

In [ ]:
def text_on_img(test_img,text,x,y):
    cv2.putText(test_img,text,(x,y),cv2.FONT_HERSHEY_DUPLEX,2,(255,0,0),2)

In [ ]:
name={1:"Me",2:"Person2",3:"Person3"}
#We are creating dictionary based on the labels and the data present in each of them.
#You can place your own pictures in folder "1"

In [ ]:
capture=cv2.VideoCapture(0) #0- for camera
print(capture.isOpened)
while(capture.isOpened):
    ret,frame=capture.read()
    
    faces_detected,gray_img=faceDetection(frame)
    #print("faces detected=",faces_detected)

    
    for face in faces_detected:
        
        (x,y,w,h)=face
        roi_gray=gray_img[y:y+h,x:x+w]
        label,confidence=face_recognizer.predict(roi_gray)
        print(label, confidence)
        draw_rect(frame,face)
        predicted_name=name[label]
        if(confidence>140):
            predicted_name="unknown"
        text_on_img(frame,predicted_name,x,y)
    
    cv2.imshow("webCam",frame)
    
    
    
    if cv2.waitKey(1)&0xff==ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

<built-in method isOpened of cv2.VideoCapture object at 0x00000124E0B27BD0>
1 65.345944928877
1 64.55851923429948
1 64.06271968696652
1 64.60645638095666
1 64.11637128148581
1 60.29507276407542
1 65.68301484289867
1 64.85463526070139
1 63.00191497412166
1 60.12978233096615
1 60.948837513942564
1 62.69011572147034
1 63.601866572495034
1 62.238281393288744
1 62.238281393288744
1 62.38863400600885
1 61.88678818789789
1 64.48416486607823
1 60.925074987650675
1 57.22527803154501
1 57.22527803154501
1 61.1038852727326
1 60.371577558861844
1 62.86649200760503
1 61.47326816235173
1 59.70715717652232
1 59.70715717652232
1 63.29266915602764
1 63.019382891621575
1 60.06162406803176
1 61.574744768218196
1 61.481456339821136
1 57.82509918602266
1 58.96866358410007
1 58.96866358410007
1 61.93289265402084
1 61.598698681319014
1 62.78159206422246
1 61.94311200440779
1 61.94311200440779
1 63.26812565372314
1 60.24843104802514
1 63.228649273102405
1 61.29195798225177
1 61.29195798225177
1 61.83471300012